In [ ]:
using JuMP
using GLPK
using ECOS
using MAT
using SparseArrays
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using DelimitedFiles
using COSMO
using CSV
using DataFrames
using DelimitedFiles

In [ ]:
vars = matread("R4T3.mat")
S = vars["S"]
U = vars["U"]
L = vars["L"]
lambda = vars["lambda"]
n,d = size(S)
print(size(S))
print(size(L))

n,d = size(S) # n=10 d=100
d,k = size(L) # k = 5

print("done")

In [ ]:
X = zeros(size(L)[1],size(L)[2]);

n,d = size(S)

for j in 1:size(X)[2]
    model = Model(ECOS.Optimizer)
    @variable(model, L[i,j] <= x[i = 1:d] <= U[i,j],start=X[i,j])
    @variable(model, y_abs[i = 1:n])
    @constraint(model,y_abs .>= S*x)
    @constraint(model,y_abs .>= -S*x)
    @variable(model, x_abs[1:d])
    @constraint(model, x_abs .>= x)
    @constraint(model, x_abs .>= -x)
    @objective(model, Min, (1/lambda)*sum(x_abs[i] for i in 1:d) + sum(y_abs[i] for i in 1:n));
    set_silent(model)
    optimize!(model)
    x = value.(x);
    X[:,j] = x;
    print(j)
    print(" ,")
end

In [ ]:
writedlm("output.txt", Matrix(reshape(value.(X),(d,k))), ',')